In [28]:
import pandas as pd

# Correct Saved TSV Pheno Files

In [29]:
df_Sex = pd.read_csv('/home/mabdel03/data/files/Isolation_Genetics/GWAS/Sex_Phenos/Pheno_Files/Sex/phenotype_31.tsv', sep='\t')

In [30]:
df_ForcepsMinorMD = pd.read_csv('/home/mabdel03/data/files/Isolation_Genetics/GWAS/MRI_Phenos/Pheno_Files/ForcepsMinorMD/phenotype_25526.tsv', sep='\t')

In [31]:
df_Loneliness = pd.read_csv('/home/mabdel03/data/files/Isolation_Genetics/GWAS/Isolation_Phenos/Pheno_Files/Loneliness/phenotype_2020.tsv', sep='\t')

In [32]:
df_AbilityToConfide = pd.read_csv('/home/mabdel03/data/files/Isolation_Genetics/GWAS/Isolation_Phenos/Pheno_Files/AbilityToConfide/phenotype_2110.tsv', sep='\t')

In [33]:
df_FreqVisit = pd.read_csv('/home/mabdel03/data/files/Isolation_Genetics/GWAS/Run2_Phenos/Create_Phenos/FreqVisits/FreqVisit.tsv', sep='\t')

In [34]:
df_NumHousehold = pd.read_csv('/home/mabdel03/data/files/Isolation_Genetics/GWAS/Run2_Phenos/Create_Phenos/NumHousehold/NumHousehold.tsv', sep='\t')

In [35]:
df_list = [(df_Sex, 'df_Sex'), (df_ForcepsMinorMD, 'df_ForcepsMinorMD'), (df_Loneliness, 'df_Loneliness'), (df_AbilityToConfide, 'df_AbilityToConfide'), (df_FreqVisit, 'df_FreqVisit'), (df_NumHousehold, 'df_NumHousehold')]

In [36]:
corrected_dfs = {}

In [37]:
for df, name in df_list:
    FID = []
    IID = []
    Phenotype = []
    for val in df['FID']:
        split = val.split(' ')
        FID.append(split[0])
        IID.append(split[1])
        Phenotype.append(split[2])
    corrected_dfs[f'corrected_{name.split('_')[1]}'] = pd.DataFrame(data={'FID': FID, 'IID': IID, name.split('_')[1]: Phenotype})

In [38]:
from functools import reduce

dfs_list = []

for key, df in corrected_dfs.items():
    # 1) Extract the phenotype from the key, e.g. "df_BMI" -> "BMI"
    phenotype = key.split('_', 1)[1]  # split "df_BMI" -> ["df", "BMI"]

    # 2) Rename the third column (the phenotype values) to this phenotype
    df = df.rename(columns={df.columns[2]: phenotype})

    # Collect them in a list
    dfs_list.append(df)

# 3) Iteratively merge them on ['FID','IID'] with how='outer' (keeping all)
master_df = reduce(
    lambda left, right: pd.merge(left, right, on=['FID','IID'], how='outer'),
    dfs_list
)

In [39]:
master_df

,FID,IID,Sex,ForcepsMinorMD,Loneliness,AbilityToConfide,FreqVisit,NumHousehold
0,1000011,513,1,NaN,0,3,3,2
1,1000026,2622,0,NaN,0,5,4,2
2,1000032,602,0,NaN,0,3,4,1
3,1000044,506,1,0.000833464,0,5,2,1
4,1000058,467,1,NaN,0,0,2,2
...,...,...,...,...,...,...,...,...
502352,6026407,618,0,NaN,1,3,2,1
502353,6026419,251,0,NaN,0,5,3,3
502354,6026424,NA,1,NaN,NaN,NaN,NA,NA
502355,6026436,707,0,NaN,NaN,NaN,NA,NA


In [40]:
comp = []
for idx in master_df.index:
    # note: use Python’s `or` and parentheses
    if (master_df.loc[idx,'NumHousehold'] == '1' and master_df.loc[idx,'FreqVisit'] == '7') \
       or (master_df.loc[idx,'NumHousehold'] == '1' and master_df.loc[idx,'FreqVisit'] == '6'):
        comp.append('2')
    elif master_df.loc[idx,'NumHousehold'] == 'NA' or master_df.loc[idx,'FreqVisit'] == 'NA':
        comp.append('NA')
    else:
        comp.append('1')
master_df['FreqSoc'] = comp


In [41]:
master_df['IID'] = master_df['FID'][:]

In [42]:
master_df.fillna("NA", inplace=True)

In [43]:
master_df

,FID,IID,Sex,ForcepsMinorMD,Loneliness,AbilityToConfide,FreqVisit,NumHousehold,FreqSoc
0,1000011,1000011,1,NA,0,3,3,2,1
1,1000026,1000026,0,NA,0,5,4,2,1
2,1000032,1000032,0,NA,0,3,4,1,1
3,1000044,1000044,1,0.000833464,0,5,2,1,1
4,1000058,1000058,1,NA,0,0,2,2,1
...,...,...,...,...,...,...,...,...,...
502352,6026407,6026407,0,NA,1,3,2,1,1
502353,6026419,6026419,0,NA,0,5,3,3,1
502354,6026424,6026424,1,NA,NA,NA,NA,NA,NA
502355,6026436,6026436,0,NA,NA,NA,NA,NA,NA


In [44]:
Conf_Binary = []
for idx in master_df.index:
    # note: use Python’s `or` and parentheses
    if master_df.loc[idx,'AbilityToConfide'] == '0':
        Conf_Binary.append('2')
    elif master_df.loc[idx, 'AbilityToConfide'] == 'NA':
        Conf_Binary.append('NA')
    else:
        Conf_Binary.append('1')
master_df['AbilityToConfide'] = Conf_Binary


In [45]:
master_df.columns

Index(['FID', 'IID', 'Sex', 'ForcepsMinorMD', 'Loneliness', 'AbilityToConfide',
       'FreqVisit', 'NumHousehold', 'FreqSoc'],
      dtype='object')

In [46]:
master_df.head

<bound method NDFrame.head of             FID      IID Sex ForcepsMinorMD Loneliness AbilityToConfide  \
0       1000011  1000011   1             NA          0                1   
1       1000026  1000026   0             NA          0                1   
2       1000032  1000032   0             NA          0                1   
3       1000044  1000044   1    0.000833464          0                1   
4       1000058  1000058   1             NA          0                2   
...         ...      ...  ..            ...        ...              ...   
502352  6026407  6026407   0             NA          1                1   
502353  6026419  6026419   0             NA          0                1   
502354  6026424  6026424   1             NA         NA               NA   
502355  6026436  6026436   0             NA         NA               NA   
502356  6026448  6026448   1             NA          0                1   

       FreqVisit NumHousehold FreqSoc  
0              3            2

In [47]:
master_df

,FID,IID,Sex,ForcepsMinorMD,Loneliness,AbilityToConfide,FreqVisit,NumHousehold,FreqSoc
0,1000011,1000011,1,NA,0,1,3,2,1
1,1000026,1000026,0,NA,0,1,4,2,1
2,1000032,1000032,0,NA,0,1,4,1,1
3,1000044,1000044,1,0.000833464,0,1,2,1,1
4,1000058,1000058,1,NA,0,2,2,2,1
...,...,...,...,...,...,...,...,...,...
502352,6026407,6026407,0,NA,1,1,2,1,1
502353,6026419,6026419,0,NA,0,1,3,3,1
502354,6026424,6026424,1,NA,NA,NA,NA,NA,NA
502355,6026436,6026436,0,NA,NA,NA,NA,NA,NA


In [48]:
master_df['Loneliness'] = master_df['Loneliness'].replace({
    '0': '1',
    '1': '2'
})

In [49]:
master_df

,FID,IID,Sex,ForcepsMinorMD,Loneliness,AbilityToConfide,FreqVisit,NumHousehold,FreqSoc
0,1000011,1000011,1,NA,1,1,3,2,1
1,1000026,1000026,0,NA,1,1,4,2,1
2,1000032,1000032,0,NA,1,1,4,1,1
3,1000044,1000044,1,0.000833464,1,1,2,1,1
4,1000058,1000058,1,NA,1,2,2,2,1
...,...,...,...,...,...,...,...,...,...
502352,6026407,6026407,0,NA,2,1,2,1,1
502353,6026419,6026419,0,NA,1,1,3,3,1
502354,6026424,6026424,1,NA,NA,NA,NA,NA,NA
502355,6026436,6026436,0,NA,NA,NA,NA,NA,NA


In [50]:
master_df.to_csv('/home/mabdel03/data/files/Isolation_Genetics/GWAS/Run2_Phenos/Phenotype_Formatting/SocialIsolation.tsv.gz',
          sep='\t', index=False, compression='gzip')

# Stratification #1: All Loneliness Phenotypes Based On Sex

In [12]:
SI_df = master_df[['FID', 'IID', 'Loneliness', 'LeisureSocialActivities', 'AbilityToConfide']]

In [13]:
SI_df['IID'] = SI_df['FID'][:]

/tmp/ipykernel_126176/1536385202.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SI_df['IID'] = SI_df['FID'][:]


In [14]:
SI_df

,FID,IID,Loneliness,LeisureSocialActivities,AbilityToConfide
0,1000011,1000011,0,3,3
1,1000026,1000026,0,1,5
2,1000032,1000032,0,-7,3
3,1000044,1000044,0,1,5
4,1000058,1000058,0,5,0
...,...,...,...,...,...
502352,6026407,6026407,1,3,3
502353,6026419,6026419,0,1,5
502354,6026424,6026424,NaN,NaN,NaN
502355,6026436,6026436,NaN,NaN,NaN


In [15]:
SI_df.fillna("NA", inplace=True)

/tmp/ipykernel_126176/1854970226.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SI_df.fillna("NA", inplace=True)


In [17]:
len(SI_df['Loneliness'])

502357

In [18]:
len(SI_df.dropna(subset='Loneliness')['Loneliness'])

502357

In [16]:
SI_df

,FID,IID,Loneliness,LeisureSocialActivities,AbilityToConfide
0,1000011,1000011,0,3,3
1,1000026,1000026,0,1,5
2,1000032,1000032,0,-7,3
3,1000044,1000044,0,1,5
4,1000058,1000058,0,5,0
...,...,...,...,...,...
502352,6026407,6026407,1,3,3
502353,6026419,6026419,0,1,5
502354,6026424,6026424,NA,NA,NA
502355,6026436,6026436,NA,NA,NA


In [ ]:
SI_counts = pd.DataFrame(data={'': ['Loneliness', 'LeisureSocialActivities', 'AbilityToConfide'], })

In [27]:
SI_df.to_csv('/home/mabdel03/data/files/Isolation_Genetics/GWAS/Scripts/ukb21942/pheno/loneliness_NoMR.tsv.gz',
          sep='\t', index=False, compression='gzip')

## Sex Stratify

In [21]:
male_master = master_df[master_df['Sex'] == '1']

In [22]:
female_master = master_df[master_df['Sex'] == '0']

## Subset Loneliness Phenotypes

In [23]:
male_Loneliness = male_master[['FID', 'IID', 'Loneliness']]

In [24]:
female_Loneliness = female_master[['FID', 'IID', 'Loneliness']]

In [25]:
male_LeisureSocialActivities = male_master[['FID', 'IID', 'LeisureSocialActivities']]

In [26]:
female_LeisureSocialActivities = female_master[['FID', 'IID', 'LeisureSocialActivities']]

In [27]:
male_AbilityToConfide = male_master[['FID', 'IID', 'AbilityToConfide']]

In [28]:
female_AbilityToConfide = female_master[['FID', 'IID', 'AbilityToConfide']]

## Save Files

In [29]:
male_Loneliness

,FID,IID,Loneliness
0,1000011,1000011,0
3,1000044,1000044,0
4,1000058,1000058,0
8,1000097,1000097,1
10,1000116,1000116,1
...,...,...,...
502346,6026345,6026345,1
502347,6026352,6026352,0
502350,6026389,6026389,0
502354,6026424,6026424,NaN


In [30]:
male_Loneliness.to_csv('/home/mabdel03/data/files/Isolation_Genetics/GWAS/Scripts/Pheno_Data/By_Sex_Loneliness/male_Loneliness.counts.tsv.gz',
                       sep='\t',
                       index=False,
                       compression='gzip')

In [31]:
female_Loneliness.to_csv('/home/mabdel03/data/files/Isolation_Genetics/GWAS/Scripts/Pheno_Data/By_Sex_Loneliness/female_Loneliness.counts.tsv.gz',
                       sep='\t',
                       index=False,
                       compression='gzip')

In [32]:
male_LeisureSocialActivities.to_csv('/home/mabdel03/data/files/Isolation_Genetics/GWAS/Scripts/Pheno_Data/By_Sex_Loneliness/male_LeisureSocialActivities.counts.tsv.gz',
                       sep='\t',
                       index=False,
                       compression='gzip')

In [33]:
female_LeisureSocialActivities.to_csv('/home/mabdel03/data/files/Isolation_Genetics/GWAS/Scripts/Pheno_Data/By_Sex_Loneliness/female_LeisureSocialActivities.counts.tsv.gz',
                       sep='\t',
                       index=False,
                       compression='gzip')

In [34]:
male_AbilityToConfide.to_csv('/home/mabdel03/data/files/Isolation_Genetics/GWAS/Scripts/Pheno_Data/By_Sex_Loneliness/male_AbilityToConfide.counts.tsv.gz',
                       sep='\t',
                       index=False,
                       compression='gzip')

In [35]:
female_AbilityToConfide.to_csv('/home/mabdel03/data/files/Isolation_Genetics/GWAS/Scripts/Pheno_Data/By_Sex_Loneliness/female_AbilityToConfide.counts.tsv.gz',
                       sep='\t',
                       index=False,
                       compression='gzip')

# Stratification #2: MR Exposure/Outcome Splits

## MRI

In [72]:
male_ForcepsMinorMD = male_master.dropna(subset='ForcepsMinorMD')

In [73]:
female_ForcepsMinorMD = female_master.dropna(subset='ForcepsMinorMD')

In [74]:
male_ForcepsMinorMD = male_ForcepsMinorMD[['IID', 'FID', 'ForcepsMinorMD']]

In [76]:
female_ForcepsMinorMD = female_ForcepsMinorMD[['IID', 'FID', 'ForcepsMinorMD']]

In [77]:
male_ForcepsMinorMD.to_csv('/home/mabdel03/data/files/Isolation_Genetics/GWAS/Scripts/Pheno_Data/MR_Splits/MRI/male_ForcepsMinorMD.counts.tsv.gz',
                       sep='\t',
                       index=False,
                       compression='gzip')

In [78]:
female_ForcepsMinorMD.to_csv('/home/mabdel03/data/files/Isolation_Genetics/GWAS/Scripts/Pheno_Data/MR_Splits/MRI/female_ForcepsMinorMD.counts.tsv.gz',
                       sep='\t',
                       index=False,
                       compression='gzip')

## Isolation

In [114]:
for iid in male_master['IID']:
    print(iid)

513
506
467
350
187
237
333
668
216
516
609
1009
249
496
634
330
243
263
362
288
466
553
374
575
310
346
622
731
581
451
370
695
387
305
369
421
326
309
736
771
372
777
263
944
308
453
300
865
198
335
442
429
290
693
491
539
373
298
285
430
506
263
396
520
699
447
323
286
612
487
376
681
870
519
261
378
552
393
324
315
337
686
402
269
404
1065
516
494
359
450
511
687
356
803
326
424
919
270
532
244
1224
357
1088
354
317
344
461
1024
444
359
335
300
683
431
NA
414
525
353
373
232
688
572
1482
619
583
249
339
496
504
512
401
593
306
257
340
724
705
484
803
305
455
175
197
551
699
315
533
580
436
320
634
391
565
193
933
452
250
580
314
304
456
415
858
704
256
417
408
296
803
314
338
320
360
534
510
552
635
290
526
480
280
463
272
353
420
699
401
399
550
264
695
384
353
1335
994
359
659
232
577
440
544
392
373
633
417
693
359
388
302
720
479
424
506
444
337
511
359
594
186
432
820
398
288
539
409
311
269
750
911
534
431
511
281
464
470
270
673
416
963
283
403
293
422
647
347
337
461
479
38

In [111]:
print(male_master['IID'].dtype)
print(male_ForcepsMinorMD['IID'].dtype)


object
object


In [112]:
n_unique_master = male_master['IID'].nunique()
n_unique_forceps = male_ForcepsMinorMD['IID'].nunique()
print("Unique IIDs in male_master:", n_unique_master)
print("Unique IIDs in male_ForcepsMinorMD:", n_unique_forceps)

Unique IIDs in male_master: 1813
Unique IIDs in male_ForcepsMinorMD: 1064


In [104]:
len(male_master['IID'])

229063

In [107]:
male_ForcepsMinorMD = male_ForcepsMinorMD.dropna(subset='ForcepsMinorMD')

In [108]:
male_isolation = male_master[~male_master['IID'].isin(male_ForcepsMinorMD['IID'])]

In [109]:
len(male_ForcepsMinorMD['IID'])

20703

In [110]:
len(male_isolation['Loneliness'])

3509

In [82]:
female_isolation = female_master[~female_master['IID'].isin(female_ForcepsMinorMD['IID'])]

### Loneliness

In [83]:
male_Loneliness = male_isolation[['FID', 'IID', 'Loneliness']]

In [100]:
len(male_Loneliness)

3497

In [84]:
female_Loneliness = female_isolation[['FID', 'IID', 'Loneliness']]

In [85]:
male_Loneliness = male_Loneliness.dropna(subset='Loneliness')

In [86]:
female_Loneliness = female_Loneliness.dropna(subset='Loneliness')

In [89]:
male_Loneliness.to_csv('/home/mabdel03/data/files/Isolation_Genetics/GWAS/Scripts/Pheno_Data/MR_Splits/Isolation/male_Loneliness.counts.tsv.gz',
                       sep='\t',
                       index=False,
                       compression='gzip')

In [90]:
female_Loneliness.to_csv('/home/mabdel03/data/files/Isolation_Genetics/GWAS/Scripts/Pheno_Data/MR_Splits/Isolation/female_Loneliness.counts.tsv.gz',
                       sep='\t',
                       index=False,
                       compression='gzip')

## LeisureSocialActivities

In [92]:
male_LeisureSocialActivities = male_isolation[['FID', 'IID', 'LeisureSocialActivities']]

In [93]:
female_LeisureSocialActivities = female_isolation[['FID', 'IID', 'LeisureSocialActivities']]

In [94]:
male_LeisureSocialActivities = male_LeisureSocialActivities.dropna(subset='LeisureSocialActivities')

In [95]:
female_LeisureSocialActivities = female_LeisureSocialActivities.dropna(subset='LeisureSocialActivities')

In [98]:
len(male_LeisureSocialActivities)

3497

In [99]:
len(female_LeisureSocialActivities)

4284

In [96]:
male_LeisureSocialActivities.to_csv('/home/mabdel03/data/files/Isolation_Genetics/GWAS/Scripts/Pheno_Data/MR_Splits/Isolation/male_LeisureSocialActivities.counts.tsv.gz',
                       sep='\t',
                       index=False,
                       compression='gzip')

In [97]:
female_LeisureSocialActivities.to_csv('/home/mabdel03/data/files/Isolation_Genetics/GWAS/Scripts/Pheno_Data/MR_Splits/Isolation/female_LeisureSocialActivities.counts.tsv.gz',
                       sep='\t',
                       index=False,
                       compression='gzip')